In [6]:
import matplotlib.pyplot as plt
from tifffile import imread, imwrite
import sys
import pandas as pd
import numpy as np

sys.path.append("../../..")

from src.data.datasets import TorchMultiImageSlideDataset
from src.utils.torch.transforms import (
    ToRGBTensor,
    CustomCenteredCrop,
    CustomResize,
    CustomRandomHorizontalFlip,
    CustomRandomVerticalFlip,
    CustomCompose,
    CustomNormalize,
    CustomBinarize
)
from torchvision import transforms

In [30]:
md = pd.read_csv("/home/paysan_d/PycharmProjects/image2reg/data/experiments/image_preprocessing/rohban/specific_targets_cv_stratified/nuclei_md_test_fold_0.csv", index_col=0)

<ipython-input-30-166d29fabfcf>:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv("/home/paysan_d/PycharmProjects/image2reg/data/experiments/image_preprocessing/rohban/specific_targets_cv_stratified/nuclei_md_test_fold_0.csv", index_col=0)


In [4]:
md = md.loc[md.gene_symbol == "EMPTY"]

In [39]:
md.groupby("toxicity").gene_symbol.unique()

toxicity
PotentiallyToxic                      [CEBPA, CXXC4, ERG, JUN]
Toxic               [ATF4, BCL2L11, CASP8, ELK1, PRKACA, RHOA]
Name: gene_symbol, dtype: object

In [26]:
md.loc[md.plate==41744].well.unique()

array(['l04', 'j13', 'l09', 'l18', 'k08', 'j02', 'm11', 'o22', 'p22',
       'o23', 'p23', 'p24', 'l17', 'o01', 'o24', 'p21', 'o21', 'h07',
       'm16', 'h19', 'a01', 'a02', 'b01', 'b02', 'b14', 'c03', 'c14',
       'd08', 'e12', 'e16', 'e17', 'f13', 'g15', 'i06', 'i23'],
      dtype=object)

In [5]:
#md = md.loc[md.slide_image_name.isin(["taoe005-u2os-72h-cp-a-au00044858_a02_s5_w18fd55173-4238-452c-b231-5a4560c30084_illum_corrected.tif", "taoe005-u2os-72h-cp-a-au00044858_m10_s3_w1cad3a0c5-b3ea-464f-9cc2-ac58c5174c58_illum_corrected.tif", "taoe005-u2os-72h-cp-a-au00044858_f21_s6_w11f14aa4e-b82b-477d-8701-b87286b889f2_illum_corrected.tif", "taoe005-u2os-72h-cp-a-au00044858_b16_s5_w1e7749bb3-671e-4a9c-8d6b-875926c8f349_illum_corrected.tif"])]
md.to_csv("/home/paysan_d/Desktop/filtered_md_data_visualization2.csv")

In [16]:
dataset = TorchMultiImageSlideDataset(nuclei_image_dir="../../../data/experiments/image_preprocessing/rohban/full_pipeline/padded_nuclei", nuclei_metadata_file= "/home/paysan_d/Desktop/filtered_md_data_visualization2.csv", slide_image_dir = "../../../data/experiments/image_preprocessing/rohban/unet_segmentation/", pseudo_rgb=True)

In [20]:
nuc_transforms =  transforms.Compose([#CustomBinarize(),
                                      transforms.Resize(224)])
slide_transforms = CustomCompose([CustomBinarize(), CustomCenteredCrop(448),CustomResize(224)])

In [22]:
dataset.set_transform_pipeline([nuc_transforms, slide_transforms])

In [23]:
import os
import tifffile
output_dir = "/home/paysan_d/Desktop/images_data/nuc_bin_region"
os.makedirs(output_dir, exist_ok=True)
for i in range(100):
    sample = dataset[i]
    ids = sample["image_file"]
    nuc_img = sample["nuclei_image"]
    slide_img = sample["slide_image"]
    os.makedirs(os.path.join(output_dir, "slide"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "nuc"),exist_ok=True)
    tifffile.imwrite( os.path.join(output_dir,"nuc", ids), np.array(nuc_img))
    tifffile.imwrite( os.path.join(output_dir,"slide", ids), np.array(slide_img))